In [2]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install wikipedia

  Stored in directory: C:\Users\hp\AppData\Local\pip\Cache\wheels\87\2a\18\4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
Note: you may need to restart the kernel to use updated packages.


In [56]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

## we will build a classifier to discern homonyms, words that are spelled the same but that have different meanings. The exact use case we will explore is to discern if the word "python" refers to the programming language or the animal.

In [7]:
import spacy
import wikipedia
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [16]:
# pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz
! python -m spacy download en_core_web_sm

  Stored in directory: C:\Users\hp\AppData\Local\Temp\pip-ephem-wheel-cache-s2pliwtb\wheels\6a\47\fb\6b5a0b8906d8e8779246c67d4658fd8a544d4a03a75520197a
Successfully built en-core-web-sm
[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [27]:
import en_core_web_sm

nlp = en_core_web_sm.load()



In [25]:
## The function allows us to pass multiples pages in constructing the documents, allowing us to prevent one class of documents 
##  from dominating the corpus.

def pages_to_sentences(*pages):
    """Return a list of sentences in Wikipedia articles."""
    sentences = []
    
    for page in pages:
        p = wikipedia.page(page)
        doc = nlp(p.content)
        sentences += [sent.text for sent in doc.sents]
    
    return sentences

animal_sents = pages_to_sentences("Reticulated python", "Ball Python","Python molurus")
language_sents = pages_to_sentences("Python (programming language)")

print("That's how our corpus looks like  :  ")
print(language_sents[:5])
print(animal_sents[:5])

That's how our corpus looks like  :  
['Python is an interpreted, high-level, general-purpose programming language.', "Created by Guido van Rossum and first released in 1991, Python's design philosophy emphasizes code readability with its notable use of significant whitespace.", 'Its language constructs and object-oriented approach aim to help programmers write clear, logical code for small and large-scale projects.', 'Python is dynamically typed and garbage-collected.', 'It supports multiple programming paradigms, including structured (particularly, procedural), object-oriented, and functional programming.']
['The reticulated python (Malayopython reticulatus) is a snake species in the family Pythonidae native to South and Southeast Asia.', "It is the world's longest snake and listed as least concern on the IUCN Red List because of its wide distribution.", 'In several range countries, it is hunted for its skin, for use in traditional medicine, and for sale as a pet.', 'It is an excelle

In [26]:
print(len(language_sents))
print(len(animal_sents))

525
403


In [28]:
x_train=language_sents[:350] + animal_sents[:350]
y_train = ["language"]*len(language_sents[:350]) + ["animal"]*len(animal_sents[:350])
x_test=language_sents[350:] + animal_sents[350:]
y_test = ["language"]*len(language_sents[350:]) + ["animal"]*len(animal_sents[350:])


#### Omitting STOP Words

Words such as "the", "a", and "or" are so common throughout our corpus that they do not contribute any signal to our data set. Further, omitting these words will reduce our already high dimensional data set.

In [29]:
from spacy.lang.en import STOP_WORDS

print(type(STOP_WORDS))
STOP_WORDS_python = STOP_WORDS.union({"python"})
STOP_WORDS_python

<class 'set'>


{"'d",
 "'ll",
 "'m",
 "'re",
 "'s",
 "'ve",
 'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'both',
 'bottom',
 'but',
 'by',
 'ca',
 'call',
 'can',
 'cannot',
 'could',
 'did',
 'do',
 'does',
 'doing',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifteen',
 'fifty',
 'first',
 'five',
 'for',
 'former',
 'formerly',
 'forty',
 'four',
 'from',
 'fron

#### Applying Lemmatization

In our current analysis, words like "python" and "pythons" will be counted as separate words. We understand that they represent the same concept and want them to be treated as the same word. 

In [30]:
def lemmatizer(text):
    return [word.lemma_ for word in nlp(text)]

# we need to generate the lemmas of the stop words
stop_words_str = " ".join(STOP_WORDS) # nlp function needs a string
stop_words_lemma = set(word.lemma_ for word in nlp(stop_words_str))


                       
                       

#### Building Classifier

The model will include:
* tf-idf weighting
* stop words
* words and bigrams
* lemmatization

Applying the above techniques should result in a data set with enough signal that a machine learning model can learn from. We will use the naive Bayes model; a probabilistic model that calculates conditional probabilities using Bayes theorem.Naive Bayes is often used as benchmark model for NLP as it is quick to train.

In [31]:
tfidf_lemma = TfidfVectorizer(#max_features=100, 
                              stop_words=stop_words_lemma.union({"python"}),
                              tokenizer=lemmatizer,
                              ngram_range=(1,2))

pipe = Pipeline([('vectorizer', tfidf_lemma), ('classifier', MultinomialNB())])
pipe.fit(x_train, y_train)

print("Training accuracy: {}".format(pipe.score(x_train, y_train)))
print("Testing accuracy: {}".format(pipe.score(x_test, y_test)))


c:\users\hp\anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['-pron-', 'yourself'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Training accuracy: 0.9342857142857143
Testing accuracy: 0.7412280701754386


In [33]:
tfidf_lemma_2 = TfidfVectorizer(#max_features=100, 
                              stop_words=STOP_WORDS_python,
                              tokenizer=lemmatizer,
                              ngram_range=(1,1))

pipe = Pipeline([('vectorizer', tfidf_lemma_2), ('classifier', MultinomialNB())])
pipe.fit(x_train, y_train)

print("Training accuracy: {}".format(pipe.score(x_train, y_train)))
print("Testing accuracy: {}".format(pipe.score(x_test, y_test)))


c:\users\hp\anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", '-PRON-', 'd', 'm', 'regard', 'use', 've'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Training accuracy: 0.9285714285714286
Testing accuracy: 0.7719298245614035


In [50]:
tfidf_lemma_3 = TfidfVectorizer(max_features=600, 
                              stop_words=STOP_WORDS.union({"python"}),
                              tokenizer=lemmatizer,
                              ngram_range=(1,1))

pipe = Pipeline([('vectorizer', tfidf_lemma_3), ('classifier', MultinomialNB())])
pipe.fit(x_train, y_train)

print("Training accuracy: {}".format(pipe.score(x_train, y_train)))
print("Testing accuracy: {}".format(pipe.score(x_test, y_test)))


c:\users\hp\anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", '-PRON-', 'd', 'm', 'regard', 'use', 've'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Training accuracy: 0.91
Testing accuracy: 0.793859649122807


In [55]:
tfidf_lemma_4 = TfidfVectorizer(max_features=800, 
                              stop_words=STOP_WORDS.union({"python"}),
                              tokenizer=lemmatizer,
                              ngram_range=(1,1))

pipe = Pipeline([('vectorizer', tfidf_lemma_4), ('classifier', MultinomialNB())])
pipe.fit(x_train, y_train)

print("Training accuracy: {}".format(pipe.score(x_train, y_train)))
print("Testing accuracy: {}".format(pipe.score(x_test, y_test)))


c:\users\hp\anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", '-PRON-', 'd', 'm', 'regard', 'use', 've'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Training accuracy: 0.9128571428571428
Testing accuracy: 0.7807017543859649


#### conclusion:

Till now our tfidf_lemma_3 model is performing best out of the rest with training and testing accuracy be **91%** and **79%** respectively. We observe that by increasing the number of features from 600 to 800 the performance of our model degrades on Testing set. This could be due to Overfitting of the model because of large numbers of features. 